In [84]:
import os
import urllib
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

img_data = {}

#Requete pour recuperer les infos de l'entité donnée - Q144/Chien par defaut
def query(wd='Q144'):
     return """SELECT ?item ?itemLabel ?pic{?item wdt:P31 wd:"""+wd+""".?item wdt:P18 ?pic}limit 100"""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

#Pour creer le fichier des images s'il existe pas
def createFolder(name='images'):
    if name not in os.listdir():
        os.mkdir(name)
    else:
        print('Dossier '+name+' déjà exitant !')

#Pour verifier si un dossier est vide - evitons de retélécharger des images inutilement.
def isEmpty(name='images'):
    if len(os.listdir(name)) == 0:
        return True
    return False

#Pour ajouter des elements a la bdd
def addImg(wd):
    results = get_results(endpoint_url, query(wd))
    for result in results['results']['bindings']:
        img_data[result['item']['value']] = [result['pic']['value']]
    print(img_data)

#Pour telecharger l'image d'un element de la bd
def downloadImage(url='https://upload.wikimedia.org/wikipedia/commons/a/ac/Bundesarchiv_B_145_Bild-F051673-0059%2C_Adolf_Hitler_und_Blondi_auf_dem_Berghof_%28cropped%29.jpg', name_img='img0'):
    full_path = 'images/'+name_img+'.jpg'
    urllib.request.urlretrieve(url, full_path)

#Pour télécharger toutes les images
def downloadAllImages(wd=['Q144']):
    if isEmpty() == False:
        print('Images déjà téléchargées !')
    createFolder()
    for w in wd:
        addImg(w)
    
    
    

downloadAllImages()


Images déjà téléchargées !
Dossier images déjà exitant !
{'http://www.wikidata.org/entity/Q86516809': ['http://commons.wikimedia.org/wiki/Special:FilePath/DetailofTheImperialPrinceandhisdogNerobyJean-BaptisteCarpeaux1865MarbleMuseed%27Orsaymharrsch.jpg'], 'http://www.wikidata.org/entity/Q87783199': ['http://commons.wikimedia.org/wiki/Special:FilePath/Photograph%20of%20Thunder%2C%20dog%20star%20of%20the%20silent%20era%2C.jpeg'], 'http://www.wikidata.org/entity/Q96176417': ['http://commons.wikimedia.org/wiki/Special:FilePath/Wasao.jpg'], 'http://www.wikidata.org/entity/Q96372991': ['http://commons.wikimedia.org/wiki/Special:FilePath/20190727-DSC00957%20%2848391674722%29.jpg'], 'http://www.wikidata.org/entity/Q97280926': ['http://commons.wikimedia.org/wiki/Special:FilePath/Dylan%20%28perro%29.jpg'], 'http://www.wikidata.org/entity/Q97356971': ['http://commons.wikimedia.org/wiki/Special:FilePath/Max%20II%20with%20medal%2C%2011.24.13.jpg'], 'http://www.wikidata.org/entity/Q104870725': ['htt